# Inner join - exemples

Dans le notebook précédent, on a bâti autour de l'intuition qu'un left join permet de faire la même chose qu'un INNER, tout en gardant l'ensemble des records de la table de gauche (même ceux pour lesquels il n'y a pas de correspondance avec la table de droite)

Maintenant, on va revoir l'exemple retail, en transformant les INNER JOIN en LEFT JOIN pour observer le résultat

In [1]:
import pandas as pd
import duckdb
import io

## Data

In [2]:
# Table des commandes:
orders_data = {
    'order_id': [1, 2, 3, 4, 5],
    'customer_id': [101, 102, 103, 104, 105]
}

df_orders = pd.DataFrame(orders_data)
df_orders

,order_id,customer_id
0,1,101
1,2,102
2,3,103
3,4,104
4,5,105


In [3]:
# Table des clients
customers_data = {
    'customer_id': [101, 102, 103, 104, 105, 106],
    'customer_name': ["Toufik", "Daniel", "Tancrède", "Kaouter", "Jean-Nicolas", "David"]
}

df_customers = pd.DataFrame(customers_data)
df_customers

,customer_id,customer_name
0,101,Toufik
1,102,Daniel
2,103,Tancrède
3,104,Kaouter
4,105,Jean-Nicolas
5,106,David


In [4]:
# Table des produits
p_names = ["Laptop", "Ipad", "Livre", "Petitos"]
products_data = {
    'product_id': [101, 103, 104, 105],
    'product_name': p_names,
    'product_price': [800, 400, 30, 2]
}

df_products = pd.DataFrame(products_data)
df_products

,product_id,product_name,product_price
0,101,Laptop,800
1,103,Ipad,400
2,104,Livre,30
3,105,Petitos,2


In [5]:
# Détail des commandes
order_details_data = {
    'order_id': [1, 2, 3, 4, 5],
    'product_id': [102, 104, 101, 103, 105],
    'quantity': [2, 1, 3, 2, 1]
}

df_order_details = pd.DataFrame(order_details_data)
df_order_details

,order_id,product_id,quantity
0,1,102,2
1,2,104,1
2,3,101,3
3,4,103,2
4,5,105,1


## Left joins pour rassembler les tables

### Exercice 1: left join pour rassembler les commandes avec les détails

In [6]:
detailed_order = pd.merge(
    df_orders,
    df_order_details,
    on='order_id',
    how='left'
)
detailed_order

,order_id,customer_id,product_id,quantity
0,1,101,102,2
1,2,102,104,1
2,3,103,101,3
3,4,104,103,2
4,5,105,105,1


In [7]:
query = """
SELECT * FROM df_orders
LEFT JOIN df_order_details
USING (order_id)
"""

# on df_orders.order_id = df_order_details.order_id

duckdb.query(query)

┌──────────┬─────────────┬────────────┬──────────┐
│ order_id │ customer_id │ product_id │ quantity │
│  int64   │    int64    │   int64    │  int64   │
├──────────┼─────────────┼────────────┼──────────┤
│        1 │         101 │        102 │        2 │
│        2 │         102 │        104 │        1 │
│        3 │         103 │        101 │        3 │
│        4 │         104 │        103 │        2 │
│        5 │         105 │        105 │        1 │
└──────────┴─────────────┴────────────┴──────────┘

Si vous changez "LEFT" par "INNER", vous verrez que pour l'instant, il n'y a pas de différence.
<br />

C'est normal ! Pour chaque ligne dans la table order, il y avait une ligne correspondante dans la table order_details.

<br />

Mais ce ne sera pas toujours le cas :

<img src="images/join_me_daenerys.gif" />

### Exercice 2:  left join pour rassembler les clients avec les commandes détaillées 

In [10]:
customers_detailed_order = pd.merge(
    df_customers,
    detailed_order,
    on = 'customer_id',
    how = 'left'
)
customers_detailed_order

,customer_id,customer_name,order_id,product_id,quantity
0,101,Toufik,1.0,102.0,2.0
1,102,Daniel,2.0,104.0,1.0
2,103,Tancrède,3.0,101.0,3.0
3,104,Kaouter,4.0,103.0,2.0
4,105,Jean-Nicolas,5.0,105.0,1.0
5,106,David,NaN,NaN,NaN


In [14]:
customers_detailed_order_query = """
SELECT *
FROM df_customers
LEFT JOIN detailed_order
USING (customer_id)
"""
duckdb.sql(customers_detailed_order_query).df()

,customer_id,customer_name,order_id,product_id,quantity
0,101,Toufik,1.0,102.0,2.0
1,102,Daniel,2.0,104.0,1.0
2,103,Tancrède,3.0,101.0,3.0
3,104,Kaouter,4.0,103.0,2.0
4,105,Jean-Nicolas,5.0,105.0,1.0
5,106,David,NaN,NaN,NaN


In [15]:
# %load solutions/1exercice2_python.py
order_client = pd.merge(
    df_customers,
    detailed_order,
    on='customer_id',
    how='left'
)
order_client


,customer_id,customer_name,order_id,product_id,quantity
0,101,Toufik,1.0,102.0,2.0
1,102,Daniel,2.0,104.0,1.0
2,103,Tancrède,3.0,101.0,3.0
3,104,Kaouter,4.0,103.0,2.0
4,105,Jean-Nicolas,5.0,105.0,1.0
5,106,David,NaN,NaN,NaN


In [16]:
# %load solutions/2exercice2_sql.py
query = """
SELECT df_customers.customer_id,
customer_name,
order_id,
product_id,
quantity
FROM df_customers
LEFT JOIN detailed_order
on df_customers.customer_id = detailed_order.customer_id
"""

duckdb.query(query).df()


,customer_id,customer_name,order_id,product_id,quantity
0,101,Toufik,1.0,102.0,2.0
1,102,Daniel,2.0,104.0,1.0
2,103,Tancrède,3.0,101.0,3.0
3,104,Kaouter,4.0,103.0,2.0
4,105,Jean-Nicolas,5.0,105.0,1.0
5,106,David,NaN,NaN,NaN


Ici, utiliser un left join nous a permis de voir que David n'a rien commandé ce mois-ci ! <br />
C'est le moment de lui envoyer un email avec une promotion ;)

### Exercice 3: inner join pour rassembler les commandes client détaillées avec les produits

In [28]:
order_w_products = pd.merge(
    order_client,
    df_products,
    on='product_id',
    how='left'
)
order_w_products

,customer_id,customer_name,order_id,product_id,quantity,product_name,product_price
0,101,Toufik,1.0,102.0,2.0,NaN,NaN
1,102,Daniel,2.0,104.0,1.0,Livre,30.0
2,103,Tancrède,3.0,101.0,3.0,Laptop,800.0
3,104,Kaouter,4.0,103.0,2.0,Ipad,400.0
4,105,Jean-Nicolas,5.0,105.0,1.0,Petitos,2.0
5,106,David,NaN,NaN,NaN,NaN,NaN


In [23]:
order_products_query = """
SELECT *
FROM df_order_details
INNER JOIN df_products
USING (product_id)
"""
duckdb.sql(order_products_query).df()

,order_id,product_id,quantity,product_name,product_price
0,2,104,1,Livre,30
1,3,101,3,Laptop,800
2,4,103,2,Ipad,400
3,5,105,1,Petitos,2


In [26]:
# %load solutions/3exercice3_python.py
order_w_products = pd.merge(
    order_client,
    df_products,
    on='product_id',
    how='left'
)
order_w_products


,customer_id,customer_name,order_id,product_id,quantity,product_name,product_price
0,101,Toufik,1.0,102.0,2.0,NaN,NaN
1,102,Daniel,2.0,104.0,1.0,Livre,30.0
2,103,Tancrède,3.0,101.0,3.0,Laptop,800.0
3,104,Kaouter,4.0,103.0,2.0,Ipad,400.0
4,105,Jean-Nicolas,5.0,105.0,1.0,Petitos,2.0
5,106,David,NaN,NaN,NaN,NaN,NaN


In [34]:
query = """
SELECT * 
FROM order_client cc
LEFT JOIN df_products products
USING (product_id)
"""

duckdb.sql(query).df()


,customer_id,customer_name,order_id,product_id,quantity,product_name,product_price
0,102,Daniel,2.0,104.0,1.0,Livre,30.0
1,103,Tancrède,3.0,101.0,3.0,Laptop,800.0
2,104,Kaouter,4.0,103.0,2.0,Ipad,400.0
3,105,Jean-Nicolas,5.0,105.0,1.0,Petitos,2.0
4,101,Toufik,1.0,102.0,2.0,None,NaN
5,106,David,NaN,NaN,NaN,None,NaN


<br />
<br />
<br />

--- 

Dans le prochain notebook, on va reprendre l'exemple de la jointure des ventes sur les univers, en répondant à une demande business différente